# climate analysis in Galicia, Spain  
### NOTE: I did not add screenshots or code about data preparation or processing to avoid making this notebook too long. If you want to see this part better, go to the GitHub repository.

Introducción

Obtención de datos

Limpieza y transformación

Análisis exploratorio

Modelado (si aplica)

Conclusiones

Ideas futuras

## Motivation
I lived in Galicia for a little over two years, and I was always told that it rained a lot in this area, and even more in the past, that "the cold in Galicia is different, it gets into your bones."  
This always sparked my curiosity, asking myself: How many days does it rain a year? How cold is it? Is this "chill in your bones" due to humidity? Which city is the coldest of all? And many other questions I didn't know how to answer.  
So I decided to do this project. Its purpose is to answer several of these questions that sparked my curiosity.  

## ASK 
This project contains data from the largest cities in Galicia (Coruña, Lugo, Ourense, Vigo, Pontevedra, and Santiago de Compostela), where I will conduct data analysis from January 1, 2023, to March 1, 2025. To answer some of the aforementioned questions, I will conduct other types of studies, comparisons, and predictions.

### Questions  
- Which city has the most stable climate (least variability in temperature)?
- How are cities organized by temperature?
- How are cities organized by precipitation?
- How are cities organized by humidity?
- Which is the most extreme city (maximum and minimum temperatures furthest from the Galician average)?
- What climate trends are observed between 2023 and 2024?
- What relationships exist between temperature, humidity, and precipitation?
- What is the percentage of rainfall in Galicia? (days per year)

## Prepare  
### Data
All data was obtained from MeteoGalicia and its MeteoSIX API.
They cover the period from January 1, 2023, to March 31, 2025. Three variables of interest are included: Precipitation, Temperature, and Humidity.

### Tools
The project is largely written in Python.
The libraries used are: Pandas, OS, Streamlit, Plotly, Seaborn, Folium, among others.


### Data type
The data obtained by MeteoGalicia is provided in CSV format. They have a simple graphical interface for obtaining this data from their website. You can obtain data for up to 10 years, but only for one point at a time. Forecast data, on the other hand, is obtained from the MeteoSIX API in JSON (see the streamlit).  
### They are organized  
We have 6 tables (one for each city) with a total of 4 columns (date, humidity, precipitation, and temperature) and 821 rows (one row is equivalent to one day). This represents a total of almost 5,000 data points.
Meanwhile, for the forecast, there are 6 time tables (one for each city) and 5 columns (here, sky_state is added to the columns). This represents a total of 144 data points, considering all the tables.


## Process


In [2]:
## Motivation